In [0]:
import requests
from urllib.parse import urljoin
from datetime import datetime
from dateutil.relativedelta import relativedelta
import zipfile
import os
import pandas as pd
import io

from pyspark.sql.functions import (
    isnan,
    when,
    count,
    col,
    input_file_name,
    element_at,
    split,
    regexp_replace,
)
from pyspark.sql import functions as F

## Save Raw data

In [0]:
def download_and_extract_zip(data: datetime, destination_path: str):
    data_formatada = data.strftime("%Y%m")
    url = f"https://portaldatransparencia.gov.br/download-de-dados/servidores/{data_formatada}_Aposentados_SIAPE"

    try:
        response = requests.get(url)
        response.raise_for_status()
        zip_content = response.content

        with zipfile.ZipFile(io.BytesIO(zip_content)) as zip_file:
            for file_info in zip_file.infolist():
                with zip_file.open(file_info) as extracted_file:
                    file_content = extracted_file.read()

                    try:
                        file_text = file_content.decode('iso-8859-1')
                        dest_file_path = f"dbfs:{destination_path}{file_info.filename}"
                        dbutils.fs.put(dest_file_path, file_text, overwrite=True)
                        print(f"Arquivo '{file_info.filename}' extraído para '{destination_path}'")
                    except UnicodeDecodeError:
                        print(f"Arquivo '{file_info.filename}' não é um arquivo de texto e foi ignorado.")
        print(f"Todos os arquivos extraídos para '{destination_path}'")
    except Exception as e:
        print(f"Erro: {e}")

## Salvando os dados históricos

In [0]:
# Caso tenhamos novos arquivos eles serão carregados automaticamente na camada Landing após a execução, estamos limitando a um historico de 3 meses a partir do ultimo arquivo M-2

end_date = datetime.today() - relativedelta(months=1) # 1 mes pois assim será incluido o ultimo mes M-2 (no caso atual agosto/2024)
start_date = end_date - relativedelta(months=3)

for file_date in pd.date_range(start_date, end_date, freq="M"):
    landing_path = f"/FileStore/landing/siape/{file_date.strftime('%Y')}/{file_date.strftime('%m')}/"
    print("Downloading and extracting the file:", file_date.strftime("%Y%m"))
    download_and_extract_zip(file_date, landing_path)

Wrote 161551214 bytes.
Arquivo '202406_Cadastro.csv' extraído para '/FileStore/landing/siape/2024/06/'
Wrote 78893 bytes.
Arquivo '202406_Observacoes.csv' extraído para '/FileStore/landing/siape/2024/06/'
Wrote 131349296 bytes.
Arquivo '202406_Remuneracao.csv' extraído para '/FileStore/landing/siape/2024/06/'
Todos os arquivos extraídos para '/FileStore/landing/siape/2024/06/'
Wrote 161663012 bytes.
Arquivo '202407_Cadastro.csv' extraído para '/FileStore/landing/siape/2024/07/'
Wrote 79944 bytes.
Arquivo '202407_Observacoes.csv' extraído para '/FileStore/landing/siape/2024/07/'
Wrote 130068699 bytes.
Arquivo '202407_Remuneracao.csv' extraído para '/FileStore/landing/siape/2024/07/'
Todos os arquivos extraídos para '/FileStore/landing/siape/2024/07/'
Wrote 161089678 bytes.
Arquivo '202408_Cadastro.csv' extraído para '/FileStore/landing/siape/2024/08/'
Wrote 80096 bytes.
Arquivo '202408_Observacoes.csv' extraído para '/FileStore/landing/siape/2024/08/'
Wrote 129589163 bytes.
Arquivo '202

## Criando o schema

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS public_informations")

Out[62]: DataFrame[]

## Criando a tabela de Cadastros

In [0]:
df_cadastro_raw = spark.read.csv("dbfs:/FileStore/landing/siape/*/*/*_Cadastro.csv", sep=";", encoding="UTF-8", header=True)

rename_dict = {
    'Id_SERVIDOR_PORTAL': 'id_servidor_portal',
    'NOME': 'nome',
    'CPF': 'cpf',
    'MATRICULA': 'matricula',
    'COD_TIPO_APOSENTADORIA': 'codigo_tipo_aposentadoria',
    'TIPO_APOSENTADORIA': 'tipo_aposentadoria',
    'DATA_APOSENTADORIA': 'data_aposentadoria',
    'DESCRICAO_CARGO': 'descricao_cargo',
    'COD_UORG_LOTACAO': 'codigo_uorg_lotacao',
    'UORG_LOTACAO': 'uorg_lotacao',
    'COD_ORG_LOTACAO': 'codigo_org_lotacao',
    'ORG_LOTACAO': 'org_lotacao',
    'COD_ORGSUP_LOTACAO': 'codigo_orgsup_lotacao',
    'ORGSUP_LOTACAO': 'orgsup_lotacao',
    'COD_TIPO_VINCULO': 'codigo_tipo_vinculo',
    'TIPO_VINCULO': 'tipo_vinculo',
    'SITUACAO_VINCULO': 'situacao_vinculo',
    'REGIME_JURIDICO': 'regime_juridico',
    'JORNADA_DE_TRABALHO': 'jornada_de_trabalho',
    'DATA_INGRESSO_CARGOFUNCAO': 'data_ingresso_cargofuncao',
    'DATA_NOMEACAO_CARGOFUNCAO': 'data_nomeacao_cargofuncao',
    'DATA_INGRESSO_ORGAO': 'data_ingresso_orgao',
    'DOCUMENTO_INGRESSO_SERVICOPUBLICO': 'documento_ingresso_servico_publico',
    'DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO': 'data_diploma_ingresso_servico_publico',
    'DIPLOMA_INGRESSO_CARGOFUNCAO': 'diploma_ingresso_cargofuncao',
    'DIPLOMA_INGRESSO_ORGAO': 'diploma_ingresso_orgao',
    'DIPLOMA_INGRESSO_SERVICOPUBLICO': 'diploma_ingresso_servico_publico'
}

df_cadastro = df_cadastro_raw.select([
    F.col(c).alias(rename_dict.get(c, c)) for c in df_cadastro_raw.columns
])

df_cadastro = (
    df_cadastro
    .dropna(subset=["cpf"])
    .withColumn("cpf", F.regexp_replace("cpf", r"[^\d]", ""))
    .withColumn("matricula", F.regexp_replace("matricula", r"[^\d]", ""))
    .withColumn('nome_arquivo', F.element_at(F.split(F.input_file_name(), '/'), -1))
    .withColumn("mes_referencia", F.element_at(F.split("nome_arquivo", "_"), 1))
    .withColumn("ano", F.substring("mes_referencia", 1, 4))
    .withColumn("mes", F.substring("mes_referencia", 5, 2))
    .drop("mes_referencia")
)

df_cadastro.write \
    .partitionBy("ano", "mes") \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("public_informations.cadastros")

spark.sql("OPTIMIZE public_informations.cadastros ZORDER BY cpf")
# display(spark.sql("SELECT * FROM public_informations.cadastros LIMIT 100"))

id_servidor_portal,nome,cpf,matricula,codigo_tipo_aposentadoria,tipo_aposentadoria,data_aposentadoria,descricao_cargo,codigo_uorg_lotacao,uorg_lotacao,codigo_org_lotacao,org_lotacao,codigo_orgsup_lotacao,orgsup_lotacao,codigo_tipo_vinculo,tipo_vinculo,situacao_vinculo,regime_juridico,jornada_de_trabalho,data_ingresso_cargofuncao,data_nomeacao_cargofuncao,data_ingresso_orgao,documento_ingresso_servico_publico,data_diploma_ingresso_servico_publico,diploma_ingresso_cargofuncao,diploma_ingresso_orgao,diploma_ingresso_servico_publico,nome_arquivo,ano,mes
2191717,AARAO ANDRADE FILHO,458763,002,01,APOSENTADORIA VOLUNTARIA,01/03/2011,PROFESSOR ENS BASICO TECN TECNOLOGICO,-3,Inválido,26431,Instituto Federal do Piauí,15000,Ministério da Educação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,null,29/12/2008,000000282,07/03/1980,null,LEI,CONTRATO,202406_Cadastro.csv,2024,06
2025221,AARAO DE ANDRADE LIMA,559144,003,01,APOSENTADORIA VOLUNTARIA,12/11/2012,PROFESSOR DO MAGISTERIO SUPERIOR,-3,Inválido,26252,Universidade Federal de Campina Grande - PB,15000,Ministério da Educação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,null,10/04/2002,SN,10/03/1977,null,LEI,PORTARIA,202406_Cadastro.csv,2024,06
3594060,AARAO MOREIRA DA SILVA,924486,004,01,APOSENTADORIA VOLUNTARIA,16/05/2011,AGENTE DE SAUDE PUBLICA,-3,Inválido,25000,Ministério da Saúde,-1,Sem informação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/03/2006,null,29/06/2010,SN,06/10/1975,null,PORTARIA,CONTRATO,202406_Cadastro.csv,2024,06
87244,ABA ISRAEL COHEN PERSIANO,681016,003,01,APOSENTADORIA VOLUNTARIA,05/05/2014,PROFESSOR DO MAGISTERIO SUPERIOR,-3,Inválido,26238,Universidade Federal de Minas Gerais,15000,Ministério da Educação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,29/01/1997,null,29/03/1978,0000000SN,18/04/1974,null,PORTARIA,PORTARIA,202406_Cadastro.csv,2024,06
3336973,ABADIA BELCHIOR GOMES,078406,004,01,APOSENTADORIA VOLUNTARIA,30/11/2012,SERVENTE DE LIMPEZA,-3,Inválido,26274,Fundação Universidade Federal Uberlândia,15000,Ministério da Educação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/03/2005,null,01/02/1982,000000S/N,01/02/1982,null,CONTRATO,CONTRATO,202406_Cadastro.csv,2024,06
647322,ABADIA CANDIDA LEMES,834461,010,01,APOSENTADORIA VOLUNTARIA,29/01/2021,GUARDA DE ENDEMIAS,-3,Inválido,25000,Ministério da Saúde,-1,Sem informação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,02/01/2007,null,29/06/2010,000000019,25/05/1987,null,PORTARIA,CONTRATO,202406_Cadastro.csv,2024,06
2462025,ABADIA COSTA TAVARES FERREIRA,818812,006,01,APOSENTADORIA VOLUNTARIA,24/01/2011,AUX OPERAC SERV DIVERSOS,-3,Inválido,13300,MINISTERIO DA AGRICULTURA E PECUARIA,-1,Sem informação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2006,null,19/06/2023,000000004,01/08/1980,null,LEI,PORTARIA,202406_Cadastro.csv,2024,06
2779935,ABADIA DAS GRACAS ALVES,827786,008,01,APOSENTADORIA VOLUNTARIA,14/02/2001,TECNICO DO SEGURO SOCIAL,-3,Inválido,57202,Instituto Nacional do Seguro Social,33100,MINISTERIO DA PREVIDENCIA SOCIAL,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/10/2007,null,20/08/1981,1,20/08/1981,null,PORTARIA,PORTARIA,202406_Cadastro.csv,2024,06
17151,ABADIA DAS GRACAS COELHO,406551,008,01,APOSENTADORIA VOLUNTARIA,03/03/1995,TECNICO DO SEGURO SOCIAL,-3,Inválido,57202,Instituto Nacional do Seguro Social,33100,MINISTERIO DA PREVIDENCIA SOCIAL,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/10/2007,null,29/03/1985,84,18/11/1969,null,PORTARIA,PORTARIA,202406_Cadastro.csv,2024,06
359820,ABADIA DE FATIMA ROSA MACEDO,391966,014,01,APOSENTADORIA VOLUNTARIA,01/08/2023,PSICOLOGO-AREA,-3,Inválido,26274,Fundação Universidade Federal Uberlândia,15000,Ministério da Educação,5,Aposentadoria,APOSENTADO,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/03/2005,null,07/07/2004,755,15/07/2004,null,PORTA

## Criando a tabela de Observações

In [0]:
df_observacoes_raw = spark.read.csv("dbfs:/FileStore/landing/siape/*/*/*_Observacoes.csv", sep=";", encoding="UTF-8", header=True)

rename_dict = {
    'ANO': 'ano',
    'MES': 'mes',
    'Id_SERVIDOR_PORTAL': 'id_servidor_portal',
    'NOME': 'nome',
    'CPF': 'cpf',
    'OBSERVACAO': 'observacao',
    'filename': 'nome_arquivo',
}

df_observacoes = df_observacoes_raw.select([
    F.col(c).alias(rename_dict.get(c, c)) for c in df_observacoes_raw.columns
])

df_observacoes = (
    df_observacoes.dropna(subset=["cpf"])
    .withColumn("cpf", regexp_replace(col("cpf"), r"[^\d]", ""))
    .withColumn("nome_arquivo", element_at(split(input_file_name(), "/"), -1))
)

df_observacoes.write \
    .partitionBy("ano", "mes") \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("public_informations.observacoes")

spark.sql("OPTIMIZE public_informations.observacoes ZORDER BY cpf")
display(spark.sql("select * FROM public_informations.observacoes limit 100"))

ano,mes,id_servidor_portal,nome,cpf,observacao,nome_arquivo
2024,08,1673521,ABIMAEL ARAUJO DOS SANTOS,999362,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,1113787,ABRAHAO SEVERO RIBEIRO,178364,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,273492,ADA MARIA DE SOUZA DOERING,058880,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,2554154,ADELIA MARIA ENGRACIA GAMA DE OLIVEIRA RODRIGUES,802292,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,1287416,ADEMIR MELO SOUZA,298372,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,161537,ADESIO LESSA RODRIGUES,097392,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,3366055,ADHERBAL AUGUSTO MEIRA MATTOS,471122,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,1221610,ADILIO ALVES CANAZA,814088,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,2774974,ADILSON JORGE HERMELINO,377458,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv
2024,08,339640,ADILSON JOSE DE SOUZA,006818,SERVIDOR PERCEBE PARCELA REMUNERATORIA NAO INCIDENTE PARA CALCULO DE TETO CONSTITUCIONAL,202408_Observacoes.csv


## Criando a tabela de Remuneração

In [0]:
from pyspark.sql.types import DecimalType

decimal_precision = 15
decimal_scale = 2

df_remuneracao_raw = spark.read.csv("dbfs:/FileStore/landing/siape/*/*/*_Remuneracao.csv", sep=";", encoding="UTF-8", header=True, inferSchema=True)

rename_dict = {
    "ANO": "ano",
    "MES": "mes",
    "Id_SERVIDOR_PORTAL": "id_servidor_portal",
    "CPF": "cpf",
    "NOME": "nome",
    "REMUNERAÇÃO BÁSICA BRUTA (R$)": "remuneracao_basica_bruta_brl",
    "REMUNERAÇÃO BÁSICA BRUTA (U$)": "remuneracao_basica_bruta_usd",
    "ABATE-TETO (R$)": "abate_teto_brl",
    "ABATE-TETO (U$)": "abate_teto_usd",
    "GRATIFICAÇÃO NATALINA (R$)": "gratificacao_natalina_brl",
    "GRATIFICAÇÃO NATALINA (U$)": "gratificacao_natalina_usd",
    "ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)": "abate_teto_gratificacao_natalina_brl",
    "ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)": "abate_teto_gratificacao_natalina_usd",
    "FÉRIAS (R$)": "ferias_brl",
    "FÉRIAS (U$)": "ferias_usd",
    "OUTRAS REMUNERAÇÕES EVENTUAIS (R$)": "outras_remuneracoes_eventuais_brl",
    "OUTRAS REMUNERAÇÕES EVENTUAIS (U$)": "outras_remuneracoes_eventuais_usd",
    "IRRF (R$)": "irrf_brl",
    "IRRF (U$)": "irrf_usd",
    "PSS/RPGS (R$)": "pss_rpgs_brl",
    "PSS/RPGS (U$)": "pss_rpgs_usd",
    "DEMAIS DEDUÇÕES (R$)": "demais_deducoes_brl",
    "DEMAIS DEDUÇÕES (U$)": "demais_deducoes_usd",
    "PENSÃO MILITAR (R$)": "pensao_militar_brl",
    "PENSÃO MILITAR (U$)": "pensao_militar_usd",
    "FUNDO DE SAÚDE (R$)": "fundo_saude_brl",
    "FUNDO DE SAÚDE (U$)": "fundo_saude_usd",
    "TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)": "taxa_ocupacao_imovel_funcional_brl",
    "TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)": "taxa_ocupacao_imovel_funcional_usd",
    "REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)": "remuneracao_apos_deducoes_obrigatorias_brl",
    "REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)": "remuneracao_apos_deducoes_obrigatorias_usd",
    "VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)": "verbas_indenizatorias_pessoal_civil_brl",
    "VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)": "verbas_indenizatorias_pessoal_civil_usd",
    "VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)": "verbas_indenizatorias_pessoal_militar_brl",
    "VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)": "verbas_indenizatorias_pessoal_militar_usd",
    "VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)": "verbas_indenizatorias_desligamento_voluntario_brl",
    "VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)": "verbas_indenizatorias_desligamento_voluntario_usd",
    "TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)": "total_verbas_indenizatorias_brl",
    "TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)": "total_verbas_indenizatorias_usd",
}

decimal_cols = [
    "remuneracao_basica_bruta_brl",
    "remuneracao_basica_bruta_usd",
    "abate_teto_brl",
    "abate_teto_usd",
    "gratificacao_natalina_brl",
    "gratificacao_natalina_usd",
    "abate_teto_gratificacao_natalina_brl",
    "abate_teto_gratificacao_natalina_usd",
    "ferias_brl",
    "ferias_usd",
    "outras_remuneracoes_eventuais_brl",
    "outras_remuneracoes_eventuais_usd",
    "irrf_brl",
    "irrf_usd",
    "pss_rpgs_brl",
    "pss_rpgs_usd",
    "demais_deducoes_brl",
    "demais_deducoes_usd",
    "pensao_militar_brl",
    "pensao_militar_usd",
    "fundo_saude_brl",
    "fundo_saude_usd",
    "taxa_ocupacao_imovel_funcional_brl",
    "taxa_ocupacao_imovel_funcional_usd",
    "remuneracao_apos_deducoes_obrigatorias_brl",
    "remuneracao_apos_deducoes_obrigatorias_usd",
    "verbas_indenizatorias_pessoal_civil_brl",
    "verbas_indenizatorias_pessoal_civil_usd",
    "verbas_indenizatorias_pessoal_militar_brl",
    "verbas_indenizatorias_pessoal_militar_usd",
    "verbas_indenizatorias_desligamento_voluntario_brl",
    "verbas_indenizatorias_desligamento_voluntario_usd",
    "total_verbas_indenizatorias_brl",
    "total_verbas_indenizatorias_usd",
]

df_remuneracao = df_remuneracao_raw.select([
    F.col(c).alias(rename_dict.get(c, c)) for c in df_remuneracao_raw.columns
])

for col_name in decimal_cols:
    df_remuneracao = df_remuneracao.withColumn(col_name, regexp_replace(col(col_name), ",", ".").cast(DecimalType(decimal_precision, decimal_scale)))

df_remuneracao = (
    df_remuneracao.dropna(subset=["cpf"])
    .withColumn("cpf", regexp_replace(col("cpf"), r"[^\d]", ""))
    .withColumn("nome_arquivo", element_at(split(input_file_name(), "/"), -1))
)

df_remuneracao.write \
    .partitionBy("ano", "mes") \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("public_informations.remuneracoes")

spark.sql("OPTIMIZE public_informations.remuneracoes ZORDER BY cpf")
display(spark.sql("select * FROM public_informations.remuneracoes limit 100"))

ano,mes,id_servidor_portal,cpf,nome,remuneracao_basica_bruta_brl,remuneracao_basica_bruta_usd,abate_teto_brl,abate_teto_usd,gratificacao_natalina_brl,gratificacao_natalina_usd,abate_teto_gratificacao_natalina_brl,abate_teto_gratificacao_natalina_usd,ferias_brl,ferias_usd,outras_remuneracoes_eventuais_brl,outras_remuneracoes_eventuais_usd,irrf_brl,irrf_usd,pss_rpgs_brl,pss_rpgs_usd,demais_deducoes_brl,demais_deducoes_usd,pensao_militar_brl,pensao_militar_usd,fundo_saude_brl,fundo_saude_usd,taxa_ocupacao_imovel_funcional_brl,taxa_ocupacao_imovel_funcional_usd,remuneracao_apos_deducoes_obrigatorias_brl,remuneracao_apos_deducoes_obrigatorias_usd,verbas_indenizatorias_pessoal_civil_brl,verbas_indenizatorias_pessoal_civil_usd,verbas_indenizatorias_pessoal_militar_brl,verbas_indenizatorias_pessoal_militar_usd,verbas_indenizatorias_desligamento_voluntario_brl,verbas_indenizatorias_desligamento_voluntario_usd,total_verbas_indenizatorias_brl,total_verbas_indenizatorias_usd,nome_arquivo
2024,6,2191717,458763,AARAO ANDRADE FILHO,10826.75,0.00,0.00,0.00,5413.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1384.37,0.00,-440.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14414.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,2025221,559144,AARAO DE ANDRADE LIMA,22787.57,0.00,0.00,0.00,11393.78,0.00,0.00,0.00,0.00,0.00,140.07,0.00,-4144.66,0.00,-2364.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,27812.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,3594060,924486,AARAO MOREIRA DA SILVA,5214.16,0.00,0.00,0.00,2607.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-36.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7784.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,87244,681016,ABA ISRAEL COHEN PERSIANO,23560.45,0.00,0.00,0.00,11780.22,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-2491.83,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,32848.84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,3336973,078406,ABADIA BELCHIOR GOMES,3242.32,0.00,0.00,0.00,1621.16,0.00,0.00,0.00,0.00,0.00,321.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5184.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,647322,834461,ABADIA CANDIDA LEMES,6099.86,0.00,0.00,0.00,3049.93,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-757.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8392.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,2462025,818812,ABADIA COSTA TAVARES FERREIRA,3446.74,0.00,0.00,0.00,1723.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5170.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,2779935,827786,ABADIA DAS GRACAS ALVES,6435.17,0.00,0.00,0.00,3217.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9652.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,17151,406551,ABADIA DAS GRACAS COELHO,6102.66,0.00,0.00,0.00,3051.33,0.00,0.00,0.00,0.00,0.00,235.28,0.00,-163.64,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9225.63,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
2024,6,359820,391966,ABADIA DE FATIMA ROSA MACEDO,8995.08,0.00,0.00,0.00,4497.54,0.00,0.00,0.00,0.00,0.00,235.28,0.00,-898.73,0.00,-175.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12653.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv


## Estatisticas descritivas das tabelas

In [0]:
from pyspark.sql.functions import count, col
from pyspark.sql import DataFrame

# Função para calcular estatísticas gerais
def get_dataframe_statistics(df: DataFrame, df_name: str):
    print(f"Estatísticas do DataFrame: {df_name}")
    
    # Contar número de linhas por ano e mes
    df_grouped = df.groupBy("ano", "mes").agg(count("*").alias("num_linhas"))
    print("Contagem de linhas por ano e mes:")
    df_grouped.show()
    
    # Contar número total de linhas
    total_rows = df.count()
    print(f"Número total de linhas: {total_rows}")
    
    # Contar número total de colunas
    total_cols = len(df.columns)
    print(f"Número total de colunas: {total_cols}")

In [0]:
get_dataframe_statistics(df_cadastro, "df_cadastro")

Estatísticas do DataFrame: df_cadastro
Contagem de linhas por ano e mes:
+----+---+----------+
| ano|mes|num_linhas|
+----+---+----------+
|2024| 06|    417432|
|2024| 07|    417702|
|2024| 08|    416163|
+----+---+----------+

Número total de linhas: 1251297
Número total de colunas: 30


In [0]:
get_dataframe_statistics(df_observacoes, "df_observacoes")

Estatísticas do DataFrame: df_observacoes
Contagem de linhas por ano e mes:
+----+---+----------+
| ano|mes|num_linhas|
+----+---+----------+
|2024| 08|       510|
|2024| 07|       510|
|2024| 06|       504|
+----+---+----------+

Número total de linhas: 1524
Número total de colunas: 7


In [0]:
get_dataframe_statistics(df_remuneracao, "df_remuneracao")
display(df_remuneracao.describe())

Estatísticas do DataFrame: df_remuneracao
Contagem de linhas por ano e mes:
+----+---+----------+
| ano|mes|num_linhas|
+----+---+----------+
|2024|  6|    408982|
|2024|  7|    409260|
|2024|  8|    407745|
+----+---+----------+

Número total de linhas: 1225987
Número total de colunas: 40


summary,ano,mes,id_servidor_portal,cpf,nome,remuneracao_basica_bruta_brl,remuneracao_basica_bruta_usd,abate_teto_brl,abate_teto_usd,gratificacao_natalina_brl,gratificacao_natalina_usd,abate_teto_gratificacao_natalina_brl,abate_teto_gratificacao_natalina_usd,ferias_brl,ferias_usd,outras_remuneracoes_eventuais_brl,outras_remuneracoes_eventuais_usd,irrf_brl,irrf_usd,pss_rpgs_brl,pss_rpgs_usd,demais_deducoes_brl,demais_deducoes_usd,pensao_militar_brl,pensao_militar_usd,fundo_saude_brl,fundo_saude_usd,taxa_ocupacao_imovel_funcional_brl,taxa_ocupacao_imovel_funcional_usd,remuneracao_apos_deducoes_obrigatorias_brl,remuneracao_apos_deducoes_obrigatorias_usd,verbas_indenizatorias_pessoal_civil_brl,verbas_indenizatorias_pessoal_civil_usd,verbas_indenizatorias_pessoal_militar_brl,verbas_indenizatorias_pessoal_militar_usd,verbas_indenizatorias_desligamento_voluntario_brl,verbas_indenizatorias_desligamento_voluntario_usd,total_verbas_indenizatorias_brl,total_verbas_indenizatorias_usd,nome_arquivo
count,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987,1225987
mean,2024.0,6.998991017033623,1720066.884682301,500753.7389980481,null,10312.952126,0.000000,-9.949742,0.000000,1709.561517,0.000000,0.000000,0.000000,0.592287,0.000000,283.262592,0.000000,-1099.203345,0.000000,-598.096436,0.000000,-34.659723,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10564.459276,0.000000,282.191383,0.000000,0.000000,0.000000,0.000000,0.000000,282.191383,0.000000,null
stddev,0.0,0.8161977049891235,1070507.274118929,289038.65040489164,null,7529.335334993744,0.0,297.7316354535386,0.0,3226.8503429499992,0.0,0.0,0.0,108.27285848983452,0.0,2154.9818831867096,0.0,1664.0848344934238,0.0,1031.142435160375,0.0,608.2934090190154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7507.801809598756,0.0,1264.3772805007616,0.0,0.0,0.0,0.0,0.0,1264.3772805007616,0.0,null
min,2024,6,1002,000000,AARAO ANDRADE FILHO,0.00,0.00,-41822.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-88087.04,0.00,-94172.13,0.00,-184601.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-5520.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202406_Remuneracao.csv
max,2024,8,3963140,999993,ZYDEA GLORIA RAMOS,80731.06,0.00,0.00,0.00,30988.65,0.00,0.00,0.00,50858.43,0.00,728692.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,593941.29,0.00,246693.02,0.00,0.00,0.00,0.00,0.00,246693.02,0.00,202408_Remuneracao.csv
